# PS 88 - Lab 9 - Multivariate Regression

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats

## Part 1: Controlling with simulation

To see how controlling for confounding variables can help us estimate causal effects, we will first use some simulated data, and then move on to real data in part 2.

The political science motivation behind the simulations is the relationship between whether protest movements use violence and whether they succeed at attaining their goals. A <a href="http://cup.columbia.edu/book/why-civil-resistance-works/9780231156820">prominent finding</a> in this literature is that nonviolent movements tend to be more likely to achieve their goals. Of course, the choice to use violence is a strategic one made by members of the movement, so this may not be a causal effect.

One potential confounding variable is how repressive the government is. In particular, we may  be concerned that violence is more likely to be used when the government is more repressive, and repressive governments are less likely to grant concessions. 

Here is a simulation of this process. There  are three parameters which we will vary later:
- `b_rep` is effect of repressiveness on whether the protest  movement succeeds. We assume this is negative, meaning success is harder with a more repressive government
- `b_viol` is the effect of violence of movement success. For the first simulation, we set this to zero, meaning there is no real causal effect
- `b_rv` is the effect of repressiveness on the use of violence, which we assume is positive

The following code simulates 1000 protest movements, with three variables: `rep` is the repressiveness, `viol` is equal to 1 for a violent movement and 0 for nonviolent, and `succ` is the measure of success, which is (potentially) a function of repressiveness, the choice of violent tactics, and random noise.

We then put the variables in a pandas data frame.

In [ ]:
b_rep = -1
b_viol = 0
b_rv = 1
rep = np.random.normal(0,1,1000)
viol = 1*(b_rv*rep + np.random.normal(0,1,1000) > 0)
succ = b_rep*rep + b_viol*viol + np.random.normal(0,.3,1000)
protest = pd.DataFrame(data={'Repressive': rep, 
                             'Violent': viol, 
                             'Success': succ})
protest

Since violence is binary, we can look at the difference in average success between violent and nonviolent movements. Here is some pandas code to compute the average success of nonviolent movements. In general, if we want to pull the values of `Var1` for the subset of rows where `Var2=x` from a data frame `df`, the code is:
`df.loc[df['Var2']==x, Var1]`.

In [ ]:
np.mean(protest.loc[protest['Violent']==1, 'Success'])

**Question 1.1. Compute the average success of nonviolent movements, and the difference of means between violent and nonviolent movements.**

While we used regression for continuous variables last week, there is nothing to stop us from using it on binary (0 or 1) variables. Let's run a bivariate regression with `Success` as the dependent variable and `Violent` as the independent variable.

Recall the code to "fit" a model with independent variable IV and dependent variable DV and data frame df is `smf.ols('DV~IV', data=df).fit()`. We will save this fitted model as `succ_ols` and then use the `.summary()` function to get the output.

In [ ]:
succ_ols = smf.ols('Success~Violent', data=protest).fit()
succ_ols.summary()

We can also visualize this with a scatterplot, where we use the `sns.regplot` function to draw a best fit line too:

In [ ]:
sns.regplot(x='Violent',y='Success', data=protest, ci=0)

The best fit "line" goes from the mean of the nonviolent movements to the mean of the violent movements. Visually:

In [ ]:
sns.regplot(x='Violent',y='Success', data=protest)
plt.axhline(np.mean(protest.loc[protest['Violent']==1, 'Success']), color="red")
plt.axhline(np.mean(protest.loc[protest['Violent']==0, 'Success']), color="green")

**Question 1.2. What is the relationship between the coefficients from this regression and the average success levels of violent/nonviolent protests?**

*Words for 1.2*

We can also run regressions with dependent variables that take on values 0 or 1. This is called a "linear probability model". In this example, `Violent` is binary, and by the way we generated the data we know that higher values of `Repressive` make it more likely to be 1 versus 0. Let's check this:

In [ ]:
smf.ols('Violent~Repressive', data=protest).fit().summary()

The positive coefficient on `Repressive` confirms that we are more likely to have violence when this variable is high. Or, visually:

In [ ]:
sns.regplot(x='Repressive', y='Violent', data=protest, ci=0)

While our dependent variable really only takes on values of 0 and 1, we can interpret the predicted value here as being a prediction about the probability of violent protest. One drawback to this is that it will sometimes predict negative probabilities or probabilities greater than 1. There are some other ways to analyze data like this that doesn't make predictions outside of 0 and 1, but at the cost of being a bit more complicated to interpret.

The nice thing about this probability interpretation is that the slope then tells us how an increase in our independent variable affects the probability that the dependent variable is a 1. In this case, a 1 unit increase in our `Repressive` measure leads to a 28% increase in the probability of violent protest.

Now let's run a multivaraite regression with `Success` as the dependent variable, and `Violent` and `Repressive` as independent variables.

In [ ]:
smf.ols('Success~Violent + Repressive', data=protest).fit().summary()

**Question 1.3. How did the coefficient on `Violent` change from the bivariate regression (with just `Violent` as the independent variable) to the multivariate regression (with both `Violent` and `Repressive` as independent variables)? Why?**

*Words for 1.3*

Now let's consider a case where there is a causal effect of violence on protest. In particular, let's suppose violence leads to less success. To capture this, we can set `b_viol = -.2` and simulate data using a similar procedure. Note that in this case we explicitly compute the potential outcomes with and without violence, putting these in columns `Success0` and `Success1`.

In [ ]:
b_rep = -1
b_viol = -.2
b_rv = 1
rep = np.random.normal(0,1,1000)
viol = 1*(b_rv*rep + np.random.normal(0,1,1000) > 0)
succ0 = b_rep*rep + np.random.normal(0,.3,1000)
succ1 = succ0 + b_viol
succ = viol*succ1 + (1-viol)*succ0
protest2 = pd.DataFrame(data={'Repressive': rep, 
                                 'Violent': viol, 
                                 'Success': succ,
                                'Success0':succ0,
                                'Success1':succ1})

Here is the bivariate regression with violence predicting success (note we now use the data frame `protest2`):

In [ ]:
smf.ols('Success~Violent', data=protest2).fit().summary()

**Question 1.4. Compute the difference in means of the potential outcome of success if nonviolent (`Success0`) for movements that were violent versus not. How does this relate to the "difference of means = causal effect + selection bias" formula? (Recall the causal effect here is -.2).**

In [ ]:
# Code for 1.4

*Words for 1.4*

Here is a function which does a simulation like the ones above, which takes the parameters that represent the real causal effects as arguments, and then returns the coefficient on `Violent` in the bivariate regression and the multivariate regression (controlling for `Repressive`). 

In [ ]:
def sim_protest(b_rep, b_viol, b_rv):
    rep = np.random.normal(0,1,1000)
    viol = 1*(b_rv*rep + np.random.normal(0,1,1000) > 0)
    succ0 = b_rep*rep + np.random.normal(0,.3,1000)
    succ1 = succ0 + b_viol
    succ = viol*succ1 + (1-viol)*succ0
    protest = pd.DataFrame(data={'Repressive': rep, 
                                 'Violent': viol, 
                                 'Success': succ,
                                'Success0':succ0,
                                'Success1':succ1})
    mnocon = smf.ols('Success ~ Violent', data=protest).fit()
    mcon = smf.ols('Success ~ Violent + rep', data=protest).fit()
    return([mnocon.params[1],mcon.params[1]])

Here is a simulation with our initial parameters, with no real causal effect of violence. THis should give similar results (not exactly the same since there is some randomness in the data generation):

In [ ]:
sim_protest(b_rep=-1,b_viol=0,b_rv=1)

**Question 1.5 Use the `sim_protest` function to show that even if violence makes movements more successful, we could still observe a negative coefficient in the bivariate regression with `Success` as the dependent variable and `Violent` as the independent variable. (Hint: set `b_viol` to something between 0 and 1) Compare this with the coefficient you get from the multivaraite regression.**

*Words for 1.5*

**Question 1.6. Now run some simulations where `b_rv = 0`, meaning repressiveness does not affect the violence choice. How do the coefficients between in the bivariate and multivariate regression compare now, and why? (Hint: think about the conditions for a confounding variable to cause selection bias.)**

*Words for 1.6*

## Part 2: Voting for the ACA and 2010 Vote Share as Difference of Means/Regression
In this part we are going to look at political implications of the <a href='https://en.wikipedia.org/wiki/Affordable_Care_Act'>Affordable Care Act</a> which was passed early in Obama's first term. This was one of the more consequential pieces of domestic legislation in decades, and the Democrats lost 63 seats in the House of Representatives in the 2010 midterms after the passing of the ACA. Let's explore if there is evidence that this was driven by votes on the bill.

First, we are going to load up some data on the midterm elections in the House in 2010. The data come from <a href="https://journals.sagepub.com/doi/abs/10.1177/1532673X11433768">this paper</a>, and are stored in Stata format. We can read this in using the `read_stata` function from the pandas library.

In [ ]:
hcr_mid = pd.read_stata("hcr_midterm.dta")
hcr_mid

How someone voted on the ACA is stored in the variable `hcr_yes`, and whether they are a Republican or Democrat is in `party`. We can look at the relationship between these two with the `crosstab` function in pandas, which is like a pivot table.

In [ ]:
pd.crosstab(hcr_mid["hcr_yes"], hcr_mid["party"])

This tells us that only <a href="https://en.wikipedia.org/wiki/Joseph_Cao">one Republican</a> voted for the bill, and 39 democrats voted against it. The main comparison we want to make is whether the Democrats who voted for the bill did better or worse in the 2010 midterms than those who voted against it.

To do this, let's first subset the data to districts with democratic incumbents who ran in competitive elections (i.e., with an opponent). To do this, we get a subset of the data where two conditions are met: `hcr_mid["dem_n"] > 0` means the election was competitive (uncontested seats with only one candidate get coded as a zero in the data file) and `hcr_mid["party"]=="D")` ensures they are democrats. We use the `&` to ensure both are true.

In [ ]:
hcr_mid = hcr_mid[(hcr_mid["dem_n"] > 0) & (hcr_mid["party"]=="D")]
hcr_mid

The `dem_n` variable is the democratic candidate vote share in the 2010 midterms. Let's look at the distribution:

In [ ]:
np.mean(hcr_mid["dem_n"])

In [ ]:
sns.displot(hcr_mid["dem_n"])
plt.axvline(50)

The average D running for re-election recieved almost 60% of the vote, but quite a few lost re-election.

Now let's compare the performance of those who voted for and against the bill. First, let's create separate data frames for the Y and N voters. (Note: we could also keep them together and use the `.loc` function like above when computing means, but it's always good to see multiple ways to do the same thing!)

In [ ]:
hcr_mid_y = hcr_mid[hcr_mid["hcr_yes"] == 1]
hcr_mid_n = hcr_mid[hcr_mid["hcr_yes"] == 0]

And take the average vote shares among these two groups.

In [ ]:
mean_y = np.mean(hcr_mid_y["dem_n"])
mean_y

In [ ]:
mean_n = np.mean(hcr_mid_n["dem_n"])
mean_n

Here is the raw difference of means:

In [ ]:
dom = mean_y - mean_n
dom

**Question 2.1 Use `smf.ols` to run a bivariate regression with `dem_n` as the dependent vaiable and `hcr_yes` as the independent variable. Save the fitted model as `ols_model`, and then use `.summary()` to get the output. (Hint: see the code between questions 1.1 and 1.2 for an example of what this should look like.) Compare this with the means/difference of means above.**

*Words for 2.1*

Now let's think about some reasons why this relationship might not be causal. A major confounding variable is that those who voted Yes likely represent more liberal districts, making their re-election easier. To check this, we will look also bring Obama's 2008 vote share into our analysis.

First, let's look at the relationship between Obama's 2008 vote share and the House members 2010 vote share.

In [ ]:
sns.scatterplot(x='obama', y='dem_n', data=hcr_mid)

As we can see, there is a strong positive relationship. This shouldn't surprise us: most people vote for the same party consistently.



Let's look at the relationship between Obama vote share and voting for the ACA.

In [ ]:
sns.scatterplot(x='obama', y='hcr_yes', data=hcr_mid)

As with some of our simulated data, this looks a little goofy because the hcr_yes variable just takes on value of 0 or 1. We can still run a "linear probability model" with Obama vote share as the independent ($X$) variable and the vote as the dependent ($Y$) variable.

**Question 2.2. Use `smf.ols` to run this linear probability model and save the fitted model as `vote_model`, then use the `.summary()` function to see the output. Interpret the slope on the `obama` variable.**

Since the DV here is binary, we can interpret the slope as meaning "as Obama vote share goes up by 1%, the probability of voting for the ACA increases by 1.4%.

We can plot this prediction (note: this assumes you saved the `smf.ols` output as `vote_model`!


In [ ]:
sns.scatterplot(x='obama', y='hcr_yes', data=hcr_mid)
xrange = np.arange(30, 100)
plt.plot(xrange, vote_model.params[0] + xrange*vote_model.params[1])

Now visualize the three variable together, but doing a scatter plot of Obama vote share and 2010 Democratic vote share, with green dots for those who voted Y and orange dots for those who voted N.

In [ ]:
sns.scatterplot(x='obama', y='dem_n', data=hcr_mid_y, color="green")
sns.scatterplot(x='obama', y='dem_n', data=hcr_mid_n, color="orange")

Note the orange dots are to the left, meaning those who voted N where generally in more moderate/conservative districts.

**Question 2.3. Receate the scatterplot above, and add horizontal lines corresponing to the mean vote share of those who voted No and the mean vote share of those who voted Yes, using the `plt.axhline` function (see the last plot above question 1.2 for an example of this).**

In [1]:
# Code for 2.3

And we can look at the relationship between the vote and how liberal the district was by plotting the average Obama vote share among the Y and N districts:

In [ ]:
sns.scatterplot(x='obama', y='dem_n', data=hcr_mid_y, color="green")
sns.scatterplot(x='obama', y='dem_n', data=hcr_mid_n, color="orange")
plt.axvline(np.mean(hcr_mid_y["obama"]), color='green')
plt.axvline(np.mean(hcr_mid_n["obama"]), color='orange')

Combining what we know so far: districts where members voted Y did much better in the election, but these were also just more liberal ("safe") disticts. 

Now lets use mulitvariate regression to "control" for Obama vote share.

First, let's re-do our bivariate analysis using `smf.ols`.

In [ ]:
ols_biv = smf.ols('dem_n ~ hcr_yes', data=hcr_mid).fit()
ols_biv.summary()

**Question 2.4. Now run a multivariate regression with both `hcr_yes` and `obama` as independent variables. Save the fitted model as `ols_mv`, and then use `.summary()` to get the output. How does controlling for Obama vote share change the coefficient on the `hcr_yes` variable?**

*words for 2.4*

To make a plot, we can pull out the coefficients by adding a `.params` after the name of our fitted model (note: this assumes you named the fitted model `ols_mv`).

In [ ]:
ols_mv.params

Let's overlay the predicted value for Y and N votes as a function of Obama vote share.

In [ ]:
sns.scatterplot(x='obama', y='dem_n', data=hcr_mid_y, color="green")
sns.scatterplot(x='obama', y='dem_n', data=hcr_mid_n, color="orange")
xrange = np.arange(30, 100)
plt.plot(xrange, ols_mv.params[0] + xrange*ols_mv.params[2], color="orange")
plt.plot(xrange, ols_mv.params[0] + ols_mv.params[1] + xrange*ols_mv.params[2], color="green")

This is nice illustration of what we mean by "controlling for Obama vote share" or "holding Obama vote share constant". The model accounts for both of these variables, and so the prediction is that *for a fixed level of Obama vote share*, those who vote N do better. Visually, there are two parallel lines, and the N line is always higher. 

Knowing the Obama vote share for each Democrat in the house, the model does not predict that those who vote N will do better in general, because they tend to be in more conservative districts. But it does predict for any two members in districts where Obama got the same vote share, but one votes Y and one votes N, the one voting N will do better.

The takeaway is that the main predictor of how well D's running did in 2010 is how well Obama did in 2008. The fact that those who voted for the ACA did better overall is misleading!

This model predicts that in a counterfactual world where those who voted against the ACA voted for it, they would have done worse in their re-election bids. Conversely, those who voted for the ACA could have done better (and some likely would have been re-elected) if they voted against it. 

Some important caveats:
- This may not be the "right" model: there might be remaining confounding variables.
- We are implicitly assuming that the effect of voting for the ACA is the same for everyone, which is probably not true: in very liberal disticts voters probably would have been unappy in their representative voted no!